In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
tf.__version__

'2.7.0'

In [3]:
batch_size = 10
split = 0.3
epochs = 20

In [4]:
class fotw_dataframe():
    '''Wraps the celebA dataset, allowing an easy way to:
       - Select the features of interest,
       - Split the dataset into 'training', 'test' or 'validation' partition.
    '''
    def __init__(self,attr_file = "gender_fex_trset.csv", images_file = 'gender_smile', main_folder = "..\data\\fotw" , selected_features = [] , drop_features = [],):
        self.main_folder = main_folder
        self.images_folder   = os.path.join(main_folder, images_file)
        self.attr_path = os.path.join(self.images_folder, attr_file)
        self.features_name = []
        self.prepare(drop_features , selected_features)
        
    def prepare(self,drop_features, selected_features ):
        
        #attributes selection
        if len(selected_features) == 0  :
            self.attributes = pd.read_csv(self.attr_path)
            self.num_features = 2
        else:
            self.num_features = len(selected_features)
            selected_features = selected_features.copy()
            selected_features.append("image_name")
            self.attributes = pd.read_csv(self.attr_path,usecols=selected_features)
        
        #removing features
        if len(drop_features) != 0:
            for feature in drop_features:
                if feature in self.attributes:
                    self.attributes = self.attributes.drop(feature , axis = 1)
                    self.num_features -= 1
            
        self.attributes.set_index("image_name" , inplace = True)
        self.attributes.replace(to_replace = -1 , value = 0 , inplace = True)
        self.attributes["image_name"] = list(self.attributes.index)
        self.features_name = list(self.attributes.columns)[:-1]
        
        return self.attributes 

In [5]:
features  = ['Gender','Smile']
fotw_df = fotw_dataframe()
fotw = fotw_df.prepare(drop_features=[],selected_features= features) 
fotw

,Gender,Smile,image_name
image_name,,,
im_GenFex_00001.jpg,0,1,im_GenFex_00001.jpg
im_GenFex_00002.jpg,1,1,im_GenFex_00002.jpg
im_GenFex_00003.jpg,0,1,im_GenFex_00003.jpg
im_GenFex_00004.jpg,1,1,im_GenFex_00004.jpg
im_GenFex_00005.jpg,1,1,im_GenFex_00005.jpg
...,...,...,...
im_GenFex_09253.jpg,1,0,im_GenFex_09253.jpg
im_GenFex_09254.jpg,1,0,im_GenFex_09254.jpg
im_GenFex_09255.jpg,0,0,im_GenFex_09255.jpg


In [6]:
split_ind = int((1 - split) * fotw.shape[0])
train_df = fotw[:split_ind]
validation_df = fotw[split_ind:]
train_df,validation_df

(                     Gender  Smile           image_name
 image_name                                             
 im_GenFex_00001.jpg       0      1  im_GenFex_00001.jpg
 im_GenFex_00002.jpg       1      1  im_GenFex_00002.jpg
 im_GenFex_00003.jpg       0      1  im_GenFex_00003.jpg
 im_GenFex_00004.jpg       1      1  im_GenFex_00004.jpg
 im_GenFex_00005.jpg       1      1  im_GenFex_00005.jpg
 ...                     ...    ...                  ...
 im_GenFex_07401.jpg       0      0  im_GenFex_07401.jpg
 im_GenFex_07402.jpg       1      0  im_GenFex_07402.jpg
 im_GenFex_07403.jpg       1      0  im_GenFex_07403.jpg
 im_GenFex_07404.jpg       1      0  im_GenFex_07404.jpg
 im_GenFex_07405.jpg       0      0  im_GenFex_07405.jpg
 
 [7405 rows x 3 columns],
                      Gender  Smile           image_name
 image_name                                             
 im_GenFex_07406.jpg       1      0  im_GenFex_07406.jpg
 im_GenFex_07407.jpg       0      0  im_GenFex_07407.jpg
 im

In [7]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# augumentations for training set:
train_datagen = ImageDataGenerator(rescale=1./255)

valid_datagen = ImageDataGenerator(rescale= 1./255)

train_generator = train_datagen.flow_from_dataframe(
    dataframe=train_df,
    directory=fotw_df.images_folder,
    x_col='image_name',
    y_col=fotw_df.features_name,
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode='raw',
    shuffle = True
)

validation_generator = valid_datagen.flow_from_dataframe(dataframe=validation_df,
                                                         directory=fotw_df.images_folder,
                                                         x_col='image_name',
                                                         y_col=fotw_df.features_name,
                                                         target_size=(224, 224),
                                                         batch_size=batch_size,
                                                         class_mode='raw'
                                                        )


Found 7405 validated image filenames.
Found 1852 validated image filenames.


In [8]:
from tensorflow.keras.applications.mobilenet_v2  import MobileNetV2
from tensorflow.keras.layers import Dense , BatchNormalization , Dropout

def classifier(num_features):
    base = MobileNetV2(input_shape = (224,224,3),
                      weights = None,
                      include_top=False,
                      pooling = "avg")
    
    x = base.output
    x = Dense(1536, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.29)(x)
    top = Dense(num_features, activation='sigmoid')(x)
    classifier = tf.keras.models.Model(base.input,top)
    
    return classifier

In [9]:
num_features = len(fotw_df.features_name)
cls = classifier(num_features)
cls.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 Conv1 (Conv2D)                 (None, 112, 112, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 bn_Conv1 (BatchNormalization)  (None, 112, 112, 32  128         ['Conv1[0][0]']                  
                                )                                                             

In [10]:
cls.compile(loss='binary_crossentropy',
              optimizer= tf.keras.optimizers.Adam(learning_rate=0.0001),
              metrics='binary_accuracy')

In [11]:
from tensorflow.keras.callbacks import EarlyStopping , ModelCheckpoint

earlystop = EarlyStopping(monitor="val_binary_accuracy", patience= 3)
checkpoint_filepath = "..\data" + f"/weights-FC{fotw_df.num_features}-MobileNetV2" + "{val_binary_accuracy:.2f}.hdf5"
model_checkpoint = ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_binary_accuracy',
    mode='max',
    save_best_only=True)

In [12]:
len(validation_generator)
history = cls.fit(
    train_generator,
    epochs=epochs,
    steps_per_epoch=len(train_generator),
    validation_data=validation_generator,
    validation_steps=len(validation_generator),
    callbacks = [model_checkpoint],
    max_queue_size=1,
    verbose=1)

Epoch 1/10
741/741 [==============================] - 105s 132ms/step - loss: 0.7709 - binary_accuracy: 0.5139 - val_loss: 0.6246 - val_binary_accuracy: 0.6955
Epoch 2/10
741/741 [==============================] - 90s 122ms/step - loss: 0.7378 - binary_accuracy: 0.5224 - val_loss: 0.8667 - val_binary_accuracy: 0.1955
Epoch 3/10
741/741 [==============================] - 91s 122ms/step - loss: 0.7267 - binary_accuracy: 0.5371 - val_loss: 0.8198 - val_binary_accuracy: 0.6955
Epoch 4/10
741/741 [==============================] - 91s 123ms/step - loss: 0.7131 - binary_accuracy: 0.5555 - val_loss: 0.7934 - val_binary_accuracy: 0.1955
Epoch 5/10
741/741 [==============================] - 90s 121ms/step - loss: 0.7060 - binary_accuracy: 0.5724 - val_loss: 0.9528 - val_binary_accuracy: 0.2981
Epoch 6/10
741/741 [==============================] - 91s 123ms/step - loss: 0.6889 - binary_accuracy: 0.5828 - val_loss: 0.6337 - val_binary_accuracy: 0.7981
Epoch 7/10
741/741 [=========================

In [13]:
path = "model\\fotw"
cls.save_weights('model\\fotw_weights')

In [31]:
n_cls = classifier(40)
n_cls.load_weights("..\data\weights-FC40-MobileNetV20.87.hdf5")

[<tf.Variable 'Conv1/kernel:0' shape=(3, 3, 3, 32) dtype=float32, numpy=
 array([[[[-7.64314458e-02,  2.99846511e-02, -1.04835205e-01,
           -1.21683925e-01, -2.20051110e-02,  1.12155490e-01,
           -2.16827821e-02,  1.68360099e-01,  1.60475269e-01,
           -2.33176928e-02, -4.29368019e-02,  1.29730925e-01,
            2.20778733e-01,  1.45985410e-02,  8.80612284e-02,
           -1.93262205e-01,  4.90671024e-02, -5.94457909e-02,
            4.66535725e-02, -1.50740311e-01, -1.32853970e-01,
           -6.05782866e-02,  1.58286944e-01,  1.07952379e-01,
            6.23832867e-02,  1.14923716e-01,  4.51278780e-03,
           -1.22235283e-01, -2.51747161e-01,  3.54621634e-02,
           -7.83466622e-02,  9.75289494e-02],
          [-4.55411300e-02,  9.74391773e-02, -1.03174806e-01,
           -7.52013689e-03, -5.93158454e-02,  1.80505663e-02,
           -6.25573993e-02,  7.46877026e-03,  9.74737182e-02,
            8.32935050e-02, -1.44092798e-01,  1.67472482e-01,
            2

Epoch 1/20
741/741 [==============================] - 95s 123ms/step - loss: 0.6399 - binary_accuracy: 0.6435 - val_loss: 0.7947 - val_binary_accuracy: 0.5275
Epoch 2/20
741/741 [==============================] - 90s 121ms/step - loss: 0.6232 - binary_accuracy: 0.6555 - val_loss: 1.1969 - val_binary_accuracy: 0.3804
Epoch 3/20
741/741 [==============================] - 90s 122ms/step - loss: 0.6180 - binary_accuracy: 0.6625 - val_loss: 3.1330 - val_binary_accuracy: 0.2338
Epoch 4/20
401/741 [===============>..............] - ETA: 39s - loss: 0.5889 - binary_accuracy: 0.6835

ResourceExhaustedError: 2 root error(s) found.
  (0) RESOURCE_EXHAUSTED:  MemoryError: Unable to allocate 5.74 MiB for an array with shape (10, 224, 224, 3) and data type float32
Traceback (most recent call last):

  File "D:\Anaconda\anaconda3\lib\site-packages\tensorflow\python\ops\script_ops.py", line 275, in __call__
    ret = func(*args)

  File "D:\Anaconda\anaconda3\lib\site-packages\tensorflow\python\autograph\impl\api.py", line 649, in wrapper
    return func(*args, **kwargs)

  File "D:\Anaconda\anaconda3\lib\site-packages\tensorflow\python\data\ops\dataset_ops.py", line 992, in generator_py_func
    values = next(generator_state.get_iterator(iterator_id))

  File "D:\Anaconda\anaconda3\lib\site-packages\keras\engine\data_adapter.py", line 834, in wrapped_generator
    for data in generator_fn():

  File "D:\Anaconda\anaconda3\lib\site-packages\keras\engine\data_adapter.py", line 960, in generator_fn
    yield x[i]

  File "D:\Anaconda\anaconda3\lib\site-packages\keras_prepro